In [1]:
"""
In this notebook, the bell state circuit and deutsch algorithm will be modeled using both IBM Qiskit and pgmpy.
The results are compared at the end.
"""

'\nIn this notebook, the bell state circuit and deutsch algorithm will be modeled using both IBM Qiskit and pgmpy.\nThe results are compared at the end.\n'

In [1]:
# Imports (some imports are probably not necessary)
import numpy as np
from qiskit import IBMQ, Aer
from qiskit.providers.ibmq import least_busy
from qiskit import QuantumCircuit, assemble, transpile
from qiskit.visualization import plot_histogram
import qiskit.quantum_info as qi

# Here, we construct the bell state circuit using Qiskit
# The bit input string allows us to choose an initialization of either |00>, |01>, |10>, or |11>
inputString = "00"

circ = QuantumCircuit(2,2)
if(inputString == "01"):
    circ.x(1)
elif(inputString == "10"):
    circ.x(0)
elif(inputString == "11"):
    circ.x(0)
    circ.x(1)

# The rest of the circuit transforms the qubits into a bell state pair
circ.barrier()
circ.h(0)
circ.cx(0,1)
circ.barrier()
circ.draw()

░ ┌───┐      ░ 
q_0: ─░─┤ H ├──■───░─
      ░ └───┘┌─┴─┐ ░ 
q_1: ─░──────┤ X ├─░─
      ░      └───┘ ░ 
c: 2/════════════════

In [6]:
import sys
sys.path.insert(0, 'C:\\Users\\arpan\\Desktop\\Aresty_Quantum_Git\\pgmpy\\pgmpy')

In [9]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete.CPD import TabularCPD
import numpy as np
from pgmpy.inference import VariableElimination
from math import sqrt



bellState = BayesianNetwork([('q0m0', 'q0m1')])

if(inputString == "00"):
    cpd_q0m0 = TabularCPD(variable = 'q0m0', variable_card = 4, values = [[1/2], [0], [0], [1/2]])#, state_names={'q0m0': ['I', 'X', 'Y', 'Z']})
    
    cpd_q0m1 = TabularCPD(variable='q0m1', variable_card = 4, values = [[1, 0, 0, 0], [0, 1/sqrt(2), 1/sqrt(2), 0], [0, -1/sqrt(2), 1/sqrt(2), 0], [0, 0, 0, 0]], evidence = ['q0m0'], evidence_card = [4])
    
    bellState.add_cpds(cpd_q0m0, cpd_q0m1)
bellStateInfer = VariableElimination(bellState)
q1 = bellStateInfer.query(['q0m1'])

print(q1)

q1Vars = q1.variables
q1Values = q1.values

print(q1Vars)
print(q1Values)


ValueError: Sum or integral of conditional probabilites for node q0m0 is not equal to 1.

In [3]:
# Here, we obtain the state vector for the system after the cx gate
BellState = qi.Statevector.from_instruction(circ)
BellStateVec = BellState.__array__()
BellStateDM = (qi.DensityMatrix.from_instruction(circ)).__array__()

# The format of the state vector is [|00>, |10>, |01>, |11>]
print(BellStateVec)
print(BellStateDM)

[ 0.70710678+0.j  0.        +0.j  0.        +0.j -0.70710678+0.j]
[[ 0.5+0.j  0. +0.j  0. +0.j -0.5+0.j]
 [ 0. +0.j  0. +0.j  0. +0.j  0. +0.j]
 [ 0. +0.j  0. +0.j  0. +0.j  0. +0.j]
 [-0.5+0.j  0. +0.j  0. +0.j  0.5+0.j]]


In [4]:
# Here, we model the bell state circuit using pgmpy

# Comment out these lines
import sys
sys.path.insert(0, 'C:\\Users\\masch\\QuantumComputing\\QComp\\pgmpy')

# Imports 
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete.CPD import TabularCPD
import numpy as np
from pgmpy.inference import VariableElimination


# Initialize Bayesian Network representing bell state circuit
# Each random variable is labeled qXmY, which represents the state of qubit qX at moment Y.
# The moments match with the circuit diagram in cell 2, starting with moment 0, 1, 2, etc.
# Links between random variables are present if their is a dependence between the qubit states.
bellState = BayesianNetwork([('q0m0', 'q0m1'), ('q0m1', 'q0m2'), ('q1m0', 'q1m1'), ('q1m1', 'q1m2'), ('q0m1', 'q1m2')])

"""
Conditional Amplitude Distribution (CAD) for each qubit state
For this circuit, each CAD consists of: 
    variable name (qubit state, see above). For instance, q0m1
    variable cardinality (always 2, representing amplitudes for |0> and |1> for the qubit state of interest)
    values - consists of a list of two lists [[...],[...]], the first list corresponds to |0>, the second to |1>
        For the first list, the entries correspond to the resulting state of the qubit state of interest, given
        a certain input state of evidence variables (in this case, previous qubit states).
        
(for q0m1): If the evidence is one qubit state q0m0, the values entry is:
    [[A(|q0m1> = |0> given |q0m0> = |0>,    A(|q0m1> = |0> given |q0m0> = |1>)], [A(|q0m1> = |1> given |q0m0> = |0>,    A(|q0m1> = |1> given |q0m0> = |1>)]
    
    evidence (list of qubit states for which the current qubit state depends on)
    evidence cardinality (list of 2s, based upon how many evidence qubits there are)
"""
# Initialization
if(inputString == "00"):
    cpd_q0m0 = TabularCPD(variable = 'q0m0', variable_card = 2, values = [[1], [0]])
    cpd_q1m0 = TabularCPD(variable = 'q1m0', variable_card = 2, values = [[1], [0]])
elif(inputString == "01"):
    cpd_q0m0 = TabularCPD(variable = 'q0m0', variable_card = 2, values = [[1], [0]])
    cpd_q1m0 = TabularCPD(variable = 'q1m0', variable_card = 2, values = [[0], [1]])
elif(inputString == "10"):
    cpd_q0m0 = TabularCPD(variable = 'q0m0', variable_card = 2, values = [[0], [1]])
    cpd_q1m0 = TabularCPD(variable = 'q1m0', variable_card = 2, values = [[1], [0]])
elif(inputString == "11"):
    cpd_q0m0 = TabularCPD(variable = 'q0m0', variable_card = 2, values = [[0], [1]])
    cpd_q1m0 = TabularCPD(variable = 'q1m0', variable_card = 2, values = [[0], [1]])
elif(inputString == "custom"):
    cpd_q0m0 = TabularCPD(variable = 'q0m0', variable_card = 2, values = [[1j], [0]])
    cpd_q1m0 = TabularCPD(variable = 'q1m0', variable_card = 2, values = [[1/np.sqrt(2)], [-1j/np.sqrt(2)]])

cpd_q0m1 = TabularCPD(variable='q0m1', variable_card = 2, values = [[1/np.sqrt(2), 1/np.sqrt(2)], [1/np.sqrt(2), -1/np.sqrt(2)]], evidence = ['q0m0'], evidence_card = [2])
cpd_q1m1 = TabularCPD(variable = 'q1m1', variable_card = 2, values = [[1, 0], [0,1]], evidence = ['q1m0'], evidence_card = [2])
cpd_q1m2 = TabularCPD(variable='q1m2', variable_card = 2, values = [[1,0,0,1], [0,1,1,0]], evidence = ['q0m1', 'q1m1'], evidence_card = [2,2])
cpd_q0m2 = TabularCPD(variable='q0m2', variable_card = 2, values = [[1, 0], [0, 1]], evidence = ['q0m1'], evidence_card = [2])


# Add the CADs to the Bayesian Network, and perform variable elimination to simulate the circuit.
bellState.add_cpds(cpd_q0m0, cpd_q1m0, cpd_q0m1, cpd_q0m2, cpd_q1m2, cpd_q1m1)
bellStateInfer = VariableElimination(bellState)
q1 = bellStateInfer.query(['q1m2', 'q0m2'])

# Printing the results will display the state of the qubit system at the end of the circuit
# Note that the ordering may not be desirable. This is dealt with below.
print(q1)

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

+---------+---------+------------------+
| q0m2    | q1m2    |   phi(q0m2,q1m2) |
+=========+=========+==================+
| q0m2(0) | q1m2(0) |   0.7071+0.0000j |
+---------+---------+------------------+
| q0m2(0) | q1m2(1) |   0.0000+0.0000j |
+---------+---------+------------------+
| q0m2(1) | q1m2(0) |   0.0000+0.0000j |
+---------+---------+------------------+
| q0m2(1) | q1m2(1) |  -0.7071+0.0000j |
+---------+---------+------------------+


C:\Users\masch\Quantum Computing\QComp\pgmpy\pgmpy\extern\tabulate.py:282: ComplexWarning: Casting complex values to real discards the imaginary part
  n = conv(string)


In [5]:
# Obtain the ordering of the variables in the display above, as well as their values
q1Vars = q1.variables
q1Values = q1.values

print(q1Vars)
print(q1Values)

['q0m2', 'q1m2']
[[ 0.70710678+0.j  0.        +0.j]
 [ 0.        +0.j -0.70710678+0.j]]


In [6]:
# Here, we construct the deutsch algorithm using Qiskit
# The bit input string allows us to choose a specific function: "ab" -> f(0) = a, f(1) = b
inputString = "01"

# Circuit setup
circ = QuantumCircuit(2,2)
circ.barrier()
circ.h(0)
circ.x(1)
circ.h(1)
circ.barrier()

# Function implementation
if(inputString == "01"):
    circ.cx(0,1)
elif(inputString == "10"):
    circ.cx(0,1)
    circ.x(1)
elif(inputString == "11"):
    circ.cx(0,1)
    circ.x(1)
    circ.cx(0,1)

# End of Circuit
circ.barrier()
circ.h(0)
circ.draw()

░ ┌───┐      ░       ░ ┌───┐
q_0: ─░─┤ H ├──────░───■───░─┤ H ├
      ░ ├───┤┌───┐ ░ ┌─┴─┐ ░ └───┘
q_1: ─░─┤ X ├┤ H ├─░─┤ X ├─░──────
      ░ └───┘└───┘ ░ └───┘ ░      
c: 2/═════════════════════════════

In [7]:
# Here, we obtain the state vector for the system after the hadamard gate
dj = qi.Statevector.from_instruction(circ)
djVec = dj.__array__()
djDM = (qi.DensityMatrix.from_instruction(circ)).__array__()

# The format of the state vector is [|00>, |10>, |01>, |11>]
print(djVec)
print(djDM)

[ 0.        +0.j  0.70710678+0.j  0.        +0.j -0.70710678+0.j]
[[ 0. +0.j  0. +0.j  0. +0.j  0. +0.j]
 [ 0. +0.j  0.5+0.j  0. +0.j -0.5+0.j]
 [ 0. +0.j  0. +0.j  0. +0.j  0. +0.j]
 [ 0. +0.j -0.5+0.j  0. +0.j  0.5+0.j]]


In [8]:
# Deutsch Algorithm using pgmpy

# In this case, q1m2 = q1m3, so q1m3 is not incorporated here
dj = BayesianNetwork([('q0m0', 'q0m1'), ('q1m0', 'q1m1'), ('q0m1', 'q1m2'), ('q0m1', 'q0m2'), ('q0m2', 'q0m3'), ('q1m1', 'q1m2')])

# Function determined by inputString (see above)
if(inputString == "00"):
    cad = [[1,0,1,0],[0,1,0,1]]
elif(inputString == "01"):
    cad = [[1,0,0,1],[0,1,1,0]]
elif(inputString == "10"):
    cad = [[0,1,1,0],[1,0,0,1]]
elif(inputString == "11"):
    cad = [[0,1,0,1],[1,0,1,0]]

cpd_q0m0 = TabularCPD(variable = 'q0m0', variable_card = 2, values = [[1], [0]])
cpd_q1m0 = TabularCPD(variable = 'q1m0', variable_card = 2, values = [[0], [1]])
cpd_q0m1 = TabularCPD(variable='q0m1', variable_card = 2, values = [[1/np.sqrt(2), 1/np.sqrt(2)], [1/np.sqrt(2), 1/-np.sqrt(2)]], evidence = ['q0m0'], evidence_card = [2])
cpd_q1m1 = TabularCPD(variable='q1m1', variable_card = 2, values = [[1/np.sqrt(2), 1/np.sqrt(2)], [1/np.sqrt(2), 1/-np.sqrt(2)]], evidence = ['q1m0'], evidence_card = [2])
cpd_q0m2 = TabularCPD(variable='q0m2', variable_card = 2, values = [[1, 0], [0, 1]], evidence = ['q0m1'], evidence_card = [2])
cpd_q1m2 = TabularCPD(variable='q1m2', variable_card = 2, values = cad, evidence = ['q0m1', 'q1m1'], evidence_card = [2,2])
cpd_q0m3 = TabularCPD(variable = 'q0m3', variable_card = 2, values = [[1/np.sqrt(2), 1/np.sqrt(2)], [1/np.sqrt(2), 1/-np.sqrt(2)]], evidence = ['q0m2'], evidence_card = [2])


dj.add_cpds(cpd_q0m0, cpd_q1m0, cpd_q0m1, cpd_q0m2, cpd_q1m2, cpd_q1m1, cpd_q0m3)

djInfer = VariableElimination(dj)

q2 = djInfer.query(['q0m3','q1m2'])
print(q2)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

+---------+---------+------------------+
| q0m3    | q1m2    |   phi(q0m3,q1m2) |
+=========+=========+==================+
| q0m3(0) | q1m2(0) |   0.0000+0.0000j |
+---------+---------+------------------+
| q0m3(0) | q1m2(1) |   0.0000+0.0000j |
+---------+---------+------------------+
| q0m3(1) | q1m2(0) |   0.7071+0.0000j |
+---------+---------+------------------+
| q0m3(1) | q1m2(1) |  -0.7071+0.0000j |
+---------+---------+------------------+


In [9]:
# Obtain the ordering of the variables in the display above, as well as their values
q2Vars = q2.variables
q2Values = q2.values

print(q2Vars)
print(q2Values)

['q0m3', 'q1m2']
[[ 0.        +0.j  0.        +0.j]
 [ 0.70710678+0.j -0.70710678+0.j]]


In [10]:
def bitListBack(n):
    N = 2**n
    numList = []
    numFormat = "0" + str(n) + "b"
    for i in range(N):
        numList.append((str(format(i,numFormat))[::-1]))
    return numList

def QiskitDict(stateVec,n):
    qbits = bitListBack(n)
    QbitDict = {}
    for i in range(2**n):
        QbitDict[qbits[i]]=np.round(stateVec[i],4)
    return QbitDict

print("BellStateCircuit")
print(QiskitDict(BellStateVec,2))


BellStateCircuit
{'00': (0.7071+0j), '10': 0j, '01': 0j, '11': (-0.7071+0j)}


In [11]:
print("Deutsch Algorithm")
print(QiskitDict(djVec,2))

Deutsch Algorithm
{'00': 0j, '10': (0.7071+0j), '01': 0j, '11': (-0.7071+0j)}


In [12]:
# Obtain the ordering of the variables in the display above, as well as their values
valArr = q1.variables
valuesArr = q1.values

def create_var_order(orderArr):
    currNum = 0
    numArr = []
    for order in orderArr:
        if len(order) == 4:
            currNum = order[1]
        numArr.append(currNum)
    return numArr

def bitList(n):
    N = 2**n
    numList = []
    numFormat = "0" + str(n) + "b"
    for i in range(N):
        numList.append((str(format(i,numFormat))))
    return numList


def columnize(listOfBits):
    n = len(listOfBits[0])
    holder = []
    for i in range(n):
        col = []
        for bit in listOfBits:
            col.append(bit[i])
        holder.append(col)
    return holder

def reform():
    varOrderArr = create_var_order(valArr)
    listOfBits = bitList(len(varOrderArr))
    columns = columnize(listOfBits)
    rearrangedColumns = [None]*len(columns)
    for index, order in enumerate(varOrderArr):
        rearrangedColumns[index] = columns[int(order)]
    numOfCols = len(rearrangedColumns)
    bitStr = ""
    finalBitArr = []
    for bitIndex in range(len(rearrangedColumns[0])):
        for num in range(numOfCols):
            bitStr+=str(rearrangedColumns[num][bitIndex])
        finalBitArr.append(bitStr)
        bitStr = ""
    return finalBitArr

def createHashTable():
    resHash = {}
    bitOrder=reform()
    valuesFlat = valuesArr.flatten()
    for index, key in enumerate(bitOrder):
        resHash[key] = np.round(valuesFlat[index], 4)
    return resHash

q1PgmpyHash = createHashTable()
valArr = q2.variables
valuesArr = q2.values
q2PgmpyHash = createHashTable()

print(q1PgmpyHash == QiskitDict(BellStateVec,2))
print(q2PgmpyHash == QiskitDict(djVec,2))
print(q2PgmpyHash)
print(QiskitDict(djVec,2))
print(reform())

True
True
{'00': 0j, '01': 0j, '10': (0.7071+0j), '11': (-0.7071+0j)}
{'00': 0j, '10': (0.7071+0j), '01': 0j, '11': (-0.7071+0j)}
['00', '01', '10', '11']


In [13]:
def cpd_2_dm(obj,rvs,var):
    numQubits = len(var)
    numRVs = len(rvs)
    varOrder = obj.variables
    numVars = len(varOrder)
    qubitOrdering = []
    rvsOrdering = []
    
    for i in range(numQubits):
        v = var[i]
        j = 0
        while(j < numVars and v != varOrder[j]):
            j += 1
        qubitOrdering.append(2**(numVars - j - 1))
        
    for i in range(numRVs):
        v = rvs[i]
        j = 0
        while(j < numVars and v != varOrder[j]):
            j += 1
        rvsOrdering.append(2**(numVars - j - 1))

    vals = (obj.values).flatten()
    dm = np.zeros((2**numQubits,2**numQubits),dtype="complex_")
    numEvents = 2**numRVs
    numPermutations = 2**numQubits
    
    for i in range(numEvents):
        val1 = 0
        for j in range(numRVs):
            val1 += ((i//(2**j))%2)*rvsOrdering[numRVs - j - 1]
        arr1 = np.zeros((numPermutations,1),dtype="complex_")
        arr2 = np.zeros((1,numPermutations),dtype="complex_")
        for j in range(numPermutations):
            val2 = val1
            for k in range(numQubits):
                val2 += ((j//(2**k))%2)*qubitOrdering[numQubits - k - 1]
            arr1[j][0] = vals[val2]
            arr2[0][j] = np.conj(vals[val2])
        dm += np.matmul(arr1,arr2)
        
    return dm

In [16]:
X = cpd_2_dm(q1,[],['q1m2', 'q0m2'])
print(X)
X = cpd_2_dm(q2,[],['q1m2', 'q0m3'])
print(X)

[[ 0.5+0.j  0. +0.j  0. +0.j -0.5+0.j]
 [ 0. +0.j  0. +0.j  0. +0.j  0. +0.j]
 [ 0. +0.j  0. +0.j  0. +0.j  0. +0.j]
 [-0.5+0.j  0. +0.j  0. +0.j  0.5+0.j]]
[[ 0. +0.j  0. +0.j  0. +0.j  0. +0.j]
 [ 0. +0.j  0.5+0.j  0. +0.j -0.5+0.j]
 [ 0. +0.j  0. +0.j  0. +0.j  0. +0.j]
 [ 0. +0.j -0.5+0.j  0. +0.j  0.5+0.j]]
